In [1]:
import pandas as pd

In [2]:
def read_empipelinelogs(file_name="intake_global.log"):
    import re
    import sys
    logfile = open(file_name, "r")
    stats={}
    stages=[]
    while True:
        line=logfile.readline()
        if not line: break
        # in the first lines of the log file : "messages in usercache" : we don't use it
        # the first line for each pipeline stage contains ****** and the uuid
        # the second line after contains either the number of messages or the data processed start_ts value
        if "*****" in line:
            uuid=line.split(' ')[2][:-1]
            if "moving to long term" in line:
                line=logfile.readline()
                if "messages in response to query" not in line:
                    print("ERREUR : manque le nb de messages après stage 'moving to long term'")
                    continue
                else:
                    ts0=line.split(',')[0]
                    messages=int(line.split(' ')[2])
                    stats[uuid]={"messages":int(messages)}
                    stats[uuid]["ts0"]=ts0
                    continue
            else:
                line=logfile.readline()
                if "PipelineStages" not in line:
                    x=line.split(',')
                    print(x)
                    print("ERREUR : manque le nom de l'étape du pipeline")
                    continue
                else:
                    stage=line.split(',')[1]
                    stage=stage[stage.index('.')+1:]
                    if stage not in stages:
                        stages.append(stage)
                    ts=line.split(',')[0]
                    # in principle there is only one stage processed for each uuid so we suppose this is the case
                    stats[uuid][stage]=ts
            # pipeline processing step by step sequentially for user uuid
    return stats


In [3]:
x='2021-05-04 01:00:05'
x=pd.to_datetime(x)
y='2021-05-04 01:03:05'
y=pd.to_datetime(y)
print(y-x)

0 days 00:03:00


In [13]:
def empllogs_dict_to_df(logs):
    # this functions converts the output of the read_empipelinelogs function to a dataframe
    logs_list=[]
    for uuid in logs.keys():
        d={"uuid":uuid}
        dx=logs[uuid]
        ts0=pd.to_datetime(dx['ts0'])
        d["day"]=ts0.date()
        for y in dx.keys():
            if y=="ts0":
                continue
            if y=="messages":
                d["messages"]=dx[y]
            else:
                d[y]=(pd.to_datetime(dx[y])-ts0).total_seconds()
                # we make the time difference between one stage and the previous one (stamped at ts0)
                ts0=pd.to_datetime(dx[y])
        logs_list.append(d)
    return pd.DataFrame(logs_list)

In [14]:
logs=empllogs_dict_to_df(read_empipelinelogs())

In [15]:
logs.shape

(184, 12)

In [16]:
logs.sum()

uuid                         b612868f-31d9-4c04-9302-9e65dbf5a9bfe302a5d5-6...
messages                                                                361825
USER_INPUT_MATCH_INCOMING                                                  329
ACCURACY_FILTERING                                                          40
TRIP_SEGMENTATION                                                            4
SECTION_SEGMENTATION                                                       551
JUMP_SMOOTHING                                                              46
CLEAN_RESAMPLING                                                            25
MODE_INFERENCE                                                             175
CREATE_CONFIRMED_OBJECTS                                                   412
OUTPUT_GEN                                                                   8
dtype: object

In [99]:
logs.to_csv('logs.csv')

In [20]:
1000/60

16.666666666666668

In [25]:
logs.messages.sort_values().tail(30)

113       195
174       237
52        244
114       467
175       616
53        652
115       677
176       701
54        890
116       890
177      1072
55       1145
117      1219
178      1246
56       1347
118      1550
179      1760
57       2905
119      3229
180      4957
58       5491
120      6460
181     10819
59      11004
121     13854
182     14770
60      14849
122     31157
183    112935
61     114256
Name: messages, dtype: int64